In [8]:
import matplotlib.pyplot as plt
from pandas import read_csv
import pandas as pd #The "as" serves to give Pandas an alias since you are importing the whole library (all functions, variables, etc.).
                    #By using "as pd" you limit the scope of Pandas to just the "pd" namespace (you can still use all functions, variables, etc.)
                    #preventing naming conflicts (e.g.  if you had a variable named DataFrame in your code, it could conflict with the DataFrame 
                    #class in Pandas. However, by using the pd. prefix, you can disambiguate the two and avoid conflicts.).
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import time                                                              
import os
import numpy as np                                               
import matplotlib.pyplot as plt
import matplotlib                                                 
from sklearn.svm import SVR                                                   
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

global window
global crypto
window =  1 #No. of days to prodict ahead into the future = 1

global AllDatasets
AllDatasets = {}


global cryptos


directory = os.getcwd()

In [9]:
cryptos = ['Bitcoin', 'Ethereum', 'Tether', 'Binance', 'USD Coin'] ################# ADD CSV NAMES IN HERE #########################

In [10]:
################################# ERROR METRICS CALCULATION FUNCTION ##############################################
def evaluate_model():
    
    result['MAPE'] = mean_absolute_percentage_error(y_pred, uy_test)
    result['MSE'] = mean_squared_error(y_pred, uy_test)
    result['R2'] = r2_score(y_pred, uy_test)
    


def preprocess():
    
    global train_size
    global test_size
    global X_train
    global y_train
    global X_test
    global y_test
    global scalerX
    global scalery
    global Dataset
    global uy_test #unscaled test data
    global train_index
    
    
    #read_csv is a function imported from pandas. It reads a csv and turns X, and Y into DataFrame objects.
    Dataset = read_csv(link, usecols =[1,2,3,4,5,6,7]) #Import all columns except first one (which contains time in UNIX standard)
    Dataset[f"{window}_day_price_forecast"] = Dataset[['close']].shift(-window) #Use close values for forecast and shift up by 1 day.
    Dataset.index = pd.to_datetime(Dataset['timeDate'], format = '%Y-%m-%d') #Make the date become the index of the dataset
    
    AllDatasets[f'{crypto}'] = Dataset['close'].to_numpy()
    
    train_index = int(((len(Dataset))-window) * 0.8)
    
    X = Dataset.iloc[:,1:7]
    X = X[:-window]
    
    y = pd.DataFrame(Dataset[f'{window}_day_price_forecast'].copy()) #Dataframe is required to prevent copy from creating a pandas series
    y = y[:-window]
    
    
    
    
    #uy_test holds the unscaled y values of test dataset to be used later in evaluate_model()
    uy_test = pd.DataFrame(y.copy())
    uy_test = uy_test[train_index:] 
    uy_test = uy_test.to_numpy() #.to_numpy() returns a vertical array
    
    
    
    
    
    
    ############################################ Sequential Test Split #####################################################
    #NOTE - int() will truncate any decimal places when converting a given number into an integer (e.g. int(2.9999999) = 2, and int(2.0000001) = 2)
    
    
    
    #Create two instances of MinMaxScaler class in the range 0 to 1
    scalerX = MinMaxScaler(feature_range=(0,1))
    scalery = MinMaxScaler(feature_range=(0,1))
    
     
    X_train, X_test = scalerX.fit_transform(X)[:train_index], scalerX.fit_transform(X)[train_index:]
    y_train = scalery.fit_transform(y)[:train_index]
    
#Turns Y_train and Y_test their own 2D array into an horizontal 1D array so that they can be fed as input into the machine learning algorithms
#Y_train and Y_test on the LHS are references to the original data! (NOT new objects).     
    y_train = y_train.ravel()
    
    
    #numpy has already been given the np alias above so check if you can comment out "import numpy" and see if you can call reshape as
    #np.reshape() instead. package.function() format is also acceptable if you import a function
    

def testModel(model):
    global result
    global y_pred

    start = time.time()
    sy_pred = model.predict(X_test)
    result['Pred Time'] = (time.time())-start
    #print(f"The dimensions of sy_pred are {sy_pred.shape}, and the values of unscaled predicted y are: {sy_pred}")
   
    #reshape_sy = sy_pred.reshape(-1,1) #Reshape matrix 
    #print(f"The dimensions of reshapesy are: {reshape_sy.shape}")
    
    ####################################### UNSCALE MODEL ########################################################
    y_pred = scalery.inverse_transform(sy_pred.reshape(-1,1)) #Here -1 is a specifier for numpy to figure out (in this case) the number of rows required to make a single column array.
    
    
    evaluate_model()
    result['Crypto'] = crypto
    allCrypto.append(result)
    result = {} #Erase result dictionary to store values of next model to be trained

In [11]:
result = {} #Initialise empty dictionary
allCrypto = [] #Initialise empty list
y_plot_predvals = {} #Create an empty dictionary

In [ ]:
for crypto in cryptos:
    link = directory + '/' + crypto + '.csv'
     
    preprocess()
    
    y_plot_predvals[f'{crypto}'] = {} #When dealing with nested dictionaries, you have to create an empty dictionary for main dictionary and the nested dictionaries!!!!!!
    
    result['Model'] = 'SVR'
    model1 = SVR()
    start_time = time.time() #time() outputs the current time in UNIX standard
    model1.fit(X_train, y_train)
    result['Training Time'] = (time.time())-start_time
    testModel(model1)
    y_plot_predvals[f'{crypto}']['SVR'] = y_pred
    
    # print(f"The size of X_train is: {X_train.shape} and its size is {X_train.size}\n")
    # print(f"The size of X_test is: {X_test.shape}, and its size is {X_test.size}\n") #dimensioning required for NN algorithms (which require 3D arrays rather than 2D arrays as with SVR)
    X_train = np.reshape(X_train, (X_train.shape[0], window, X_train.shape[1])) #X_train.shape[0] = No. of rows (samples), 1 = No. of timesteps (previous day used to predict next day), X_train.shape[1] = No. of columns (No. of independent variables being used)
    X_test = np.reshape(X_test, (X_test.shape[0], window, X_test.shape[1]))
    
    
    result['Model'] = 'LSTM'
    model2 = Sequential()
    model2.add(InputLayer((window, 6))) #1 timestep, 6 variables being investigated to predict next closing price
    model2.add(LSTM(64))
    model2.add(Dense(32, activation = 'relu')) #Take away the "activation ="
    model2.add(Dense(32, activation = 'relu'))
    model2.add(Dense(1))
    print(f"Summary of model is:\n {model2.summary()}\n")
    # model2.save('LSTM_model.pb')
    #cpmod2 = ModelCheckpoint('model2/', save_best_only=True)
    model2.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001))
    start_time = time.time()
    model2.fit(X_train, y_train, epochs=50)
    result['Training Time'] = (time.time())-start_time
    testModel(model2)
    y_plot_predvals[f"{crypto}"]['LSTM'] = y_pred
    
    
    result['Model'] = 'GRU'
    model3 = Sequential()
    model3.add(InputLayer((window, 6))) #1 timestep, 6 variables being investigated to predict next closing price
    model3.add(GRU(64))
    model3.add(Dense(32, 'relu'))
    model3.add(Dense(32, 'relu'))
    model3.add(Dense(1))
    print(f"Summary of model is:\n {model3.summary()}\n")
    #cpmod3 = ModelCheckpoint('model3/', save_best_only=True)
    model3.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001)) #Problem has to do with metrics apparently!
    start_time = time.time()
    model3.fit(X_train, y_train, epochs=50)
    result['Training Time'] = (time.time())-start_time
    testModel(model3)
    y_plot_predvals[f"{crypto}"]['GRU'] = y_pred
   
    
    result['Model'] = 'CNN'
    model4 = Sequential()
    model4.add(InputLayer((window, 6)))
    model4.add(Conv1D(64, kernel_size=1))
    model4.add(Flatten())
    model4.add(Dense(8, 'relu'))
    model4.add(Dense(1, 'linear'))
    print(f"Summary of model is:\n {model4.summary()}\n")
    model4.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001)) #Problem has to do with metrics apparently!
    start_time = time.time()
    model4.fit(X_train, y_train, epochs=50)
    result['Training Time'] = (time.time())-start_time
    testModel(model4)
    y_plot_predvals[f"{crypto}"]['CNN'] = y_pred

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                18176     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 21,345
Trainable params: 21,345
Non-trainable params: 0
_________________________________________________________________


2023-05-31 09:36:38.297026: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-31 09:36:38.299691: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-31 09:36:38.301387: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Summary of model is:
 None

Epoch 1/50


2023-05-31 09:36:39.893519: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-31 09:36:39.903802: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-31 09:36:39.914443: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/10 [==============================] - 5s 12ms/step - loss: 0.1292
Epoch 2/50
10/10 [==============================] - 0s 10ms/step - loss: 0.1243
Epoch 3/50
10/10 [==============================] - 0s 13ms/step - loss: 0.1204
Epoch 4/50
10/10 [==============================] - 0s 12ms/step - loss: 0.1174
Epoch 5/50
10/10 [==============================] - 0s 18ms/step - loss: 0.1151
Epoch 6/50
10/10 [==============================] - 0s 11ms/step - loss: 0.1126
Epoch 7/50
10/10 [==============================] - 0s 4ms/step - loss: 0.1102
Epoch 8/50
10/10 [==============================] - 0s 6ms/step - loss: 0.1079
Epoch 9/50
10/10 [==============================] - 0s 4ms/step - loss: 0.1054
Epoch 10/50
10/10 [==============================] - 0s 5ms/step - loss: 0.1027
Epoch 11/50
10/10 [==============================] - 0s 11ms/step - loss: 0.0998
Epoch 12/50
10/10 [==============================] - 0s 11ms/step - loss: 0.0969
Epoch 13/50
10/10 [==============================] -

...

10/10 [==============================] - 3s 3ms/step - loss: 0.1061
Epoch 2/50
10/10 [==============================] - 0s 6ms/step - loss: 0.0942
Epoch 3/50
10/10 [==============================] - 0s 7ms/step - loss: 0.0829
Epoch 4/50
10/10 [==============================] - 0s 5ms/step - loss: 0.0728
Epoch 5/50
10/10 [==============================] - 0s 4ms/step - loss: 0.0644
Epoch 6/50
10/10 [==============================] - 0s 5ms/step - loss: 0.0564
Epoch 7/50
10/10 [==============================] - 0s 5ms/step - loss: 0.0493
Epoch 8/50
10/10 [==============================] - 0s 6ms/step - loss: 0.0428
Epoch 9/50
10/10 [==============================] - 0s 4ms/step - loss: 0.0367
Epoch 10/50
10/10 [==============================] - 0s 5ms/step - loss: 0.0310
Epoch 11/50
10/10 [==============================] - 0s 4ms/step - loss: 0.0260
Epoch 12/50
10/10 [==============================] - 0s 5ms/step - loss: 0.0217
Epoch 13/50
10/10 [==============================] - 0s 6ms/

2023-05-31 09:37:19.228206: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-31 09:37:19.229698: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-31 09:37:19.230985: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 1, 64)             448       
                                                                 
 flatten_1 (Flatten)         (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 8)                 520       
                                                                 
 dense_15 (Dense)            (None, 1)                 9         
                                                                 
Total params: 977
Trainable params: 977
Non-trainable params: 0
_________________________________________________________________
Summary of model is:
 None

Epoch 1/50
10/10 [==============================] - 1s 2ms/step - loss: 0.0082
Epoch 2/50
10/10 [==============================] - 0s 3ms/step - loss: 0.0064
Ep

2023-05-31 09:37:23.064421: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-31 09:37:23.065950: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-31 09:37:23.067170: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

 1/10 [==>...........................] - ETA: 19s - loss: 0.1327

In [ ]:
df = pd.DataFrame.from_dict(allCrypto)
df.to_csv('Metrics_Results.csv')
df

In [ ]:
#Turn datagram Dataset index (which is the date of the of each point) into a numpy array
X_TimeArray = Dataset.index.to_numpy()
X_TimeArray = X_TimeArray.astype('datetime64[D]').astype(str)

X_TimeArray = np.chararray.split(X_TimeArray, sep = '-') #Separate all strings by "-" found inside array. 
year_arr = [date[0] for date in X_TimeArray] #Choose first element (day) in sequential order from first to last list inside the array
month_arr = [date[1] for date in X_TimeArray]
day_arr = [date[2] for date in X_TimeArray]

X_TimeArray = np.char.add(np.char.add(day_arr, '-'), np.char.add(np.char.add(month_arr, '-'), year_arr))

#Turn datagram Dataset 'close' column into numpy array 
CloseArray = Dataset['close'][train_index:-window].to_numpy()

#Full X_TimeArray
spacing = 8
Full_tick_interval = len(X_TimeArray) // spacing
Full_tick_locations = np.arange(0, len(X_TimeArray), Full_tick_interval)


#Only Test Set (train_index:-window)
spacing = 8
tick_interval = len(X_TimeArray[train_index:-window]) // spacing
tick_locations = np.arange(0, len(X_TimeArray[train_index:-window]), tick_interval)

In [ ]:
##########################################  BITCOIN  CHARTS  ####################################################### 


#*********************************************  GRAPHS *********************************************************

plt.plot(X_TimeArray[:(train_index+1)], AllDatasets[cryptos[0]][:(train_index+1)], label = 'Testing Set')
plt.plot(X_TimeArray[train_index:-window], AllDatasets[cryptos[0]][train_index:-window], label = 'Training Set')
plt.plot(X_TimeArray[(-window-1):], AllDatasets[cryptos[0]][(-window-1):], label = f'window ({window} Day)')

plt.title(f'{cryptos[0]} Dataset'), 
plt.xlabel('Time (days)'), plt.ylabel('Price (USD)'), plt.grid(True), plt.legend()
plt.xticks(Full_tick_locations, X_TimeArray[Full_tick_locations], rotation = 60)
plt.savefig(f"{cryptos[0]}_Dataset", dpi=300, bbox_inches = 'tight')


plt.figure()
plt.plot(X_TimeArray[train_index:-window], AllDatasets[cryptos[0]][train_index:-window], label = 'Closing Price')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[0]]['SVR'], label = 'SVR Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[0]]['LSTM'], label = 'LSTM Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[0]]['GRU'], label = 'GRU Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[0]]['CNN'], label = 'CNN Prediction')

plt.title(f'{cryptos[0]} Closing Price Prediction') 
plt.xlabel('Time (days)'), plt.ylabel('Price (USD)'), plt.grid(True), plt.legend(loc='lower right')
plt.xticks(tick_locations, X_TimeArray[train_index:-window][tick_locations], rotation = 60)
plt.savefig(f"{cryptos[0]}_Predictions", dpi=300, bbox_inches = 'tight')


#--------------------------------------------   BAR CHARTS    -------------------------------------------------------

BarNames = ['SVR', 'LSTM', 'GRU', 'CNN']
colours = ['orange', 'green', 'darkred', 'purple']


plt.figure()
values = [allCrypto[0]['Training Time'], allCrypto[1]['Training Time'], allCrypto[2]['Training Time'], allCrypto[3]['Training Time']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[0]} Training Time')
plt.xlabel('Models')
plt.ylabel('Time (seconds)')
plt.savefig(f"{cryptos[0]}_Training_Time", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[0]['MAPE'], allCrypto[1]['MAPE'], allCrypto[2]['MAPE'], allCrypto[3]['MAPE']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[0]} Mean Absolute Percentage Error (MAPE)')
plt.xlabel('Models')
plt.ylabel('Percentage (%)')
plt.savefig(f"{cryptos[0]}_MAPE", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[0]['MSE'], allCrypto[1]['MSE'], allCrypto[2]['MSE'], allCrypto[3]['MSE']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[0]} Mean Squared Error (MSE)')
plt.xlabel('Models')
plt.ylabel('Price Squared ($USD^2$)')
plt.savefig(f"{cryptos[0]}_MSE", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[0]['R2'], allCrypto[1]['R2'], allCrypto[2]['R2'], allCrypto[3]['R2']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[0]} $R^2$ Measure')
plt.xlabel('Models')
plt.ylabel('No Units')
plt.savefig(f"{cryptos[0]}_R2", dpi=300, bbox_inches = 'tight')

In [ ]:
##########################################  ETHEREUM  CHARTS  ####################################################### 


#*********************************************  GRAPHS *********************************************************

plt.plot(X_TimeArray[:(train_index+1)], AllDatasets[cryptos[1]][:(train_index+1)], label = 'Testing Set')
plt.plot(X_TimeArray[train_index:-window], AllDatasets[cryptos[1]][train_index:-window], label = 'Training Set')
plt.plot(X_TimeArray[(-window-1):], AllDatasets[cryptos[1]][(-window-1):], label = f'window ({window} Day)')

plt.title(f'{cryptos[1]} Dataset'), 
plt.xlabel('Time (days)'), plt.ylabel('Price (USD)'), plt.grid(True), plt.legend()
plt.xticks(Full_tick_locations, X_TimeArray[Full_tick_locations], rotation = 60)
plt.savefig(f"{cryptos[1]}_Dataset", dpi=300, bbox_inches = 'tight')

plt.figure()
plt.plot(X_TimeArray[train_index:-window], AllDatasets[cryptos[1]][train_index:-window], label = 'Closing Price')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[1]]['SVR'], label = 'SVR Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[1]]['LSTM'], label = 'LSTM Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[1]]['GRU'], label = 'GRU Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[1]]['CNN'], label = 'CNN Prediction')

plt.title(f'{cryptos[1]} Closing Price Prediction') 
plt.xlabel('Time (days)'), plt.ylabel('Price (USD)'), plt.grid(True), plt.legend()
plt.xticks(tick_locations, X_TimeArray[train_index:-window][tick_locations], rotation = 60)
plt.savefig(f"{cryptos[1]}_Predictions", dpi=300, bbox_inches = 'tight')


#--------------------------------------------   BAR CHARTS    -------------------------------------------------------
BarNames = ['SVR', 'LSTM', 'GRU', 'CNN']
colours = ['orange', 'green', 'darkred', 'purple']


plt.figure()
values = [allCrypto[4]['Training Time'], allCrypto[5]['Training Time'], allCrypto[6]['Training Time'], allCrypto[7]['Training Time']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[1]} Training Time')
plt.xlabel('Models')
plt.ylabel('Time (seconds)')
plt.savefig(f"{cryptos[1]}_Training_Time", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[4]['MAPE'], allCrypto[5]['MAPE'], allCrypto[6]['MAPE'], allCrypto[7]['MAPE']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[1]} Mean Absolute Percentage Error (MAPE)')
plt.xlabel('Models')
plt.ylabel('Percentage (%)')
plt.savefig(f"{cryptos[1]}_MAPE", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[4]['MSE'], allCrypto[5]['MSE'], allCrypto[6]['MSE'], allCrypto[7]['MSE']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[1]} Mean Squared Error (MSE)')
plt.xlabel('Models')
plt.ylabel('Price Squared ($USD^2$)')
plt.savefig(f"{cryptos[1]}_MSE", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[4]['R2'], allCrypto[5]['R2'], allCrypto[6]['R2'], allCrypto[7]['R2']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[1]} $R^2$ Measure')
plt.xlabel('Models')
plt.ylabel('No Units')
plt.savefig(f"{cryptos[1]}_R2", dpi=300, bbox_inches = 'tight')

In [ ]:
##########################################  USDT CHARTS  ####################################################### 


#*********************************************  GRAPHS  *********************************************************

plt.plot(X_TimeArray[:(train_index+1)], AllDatasets[cryptos[2]][:(train_index+1)], label = 'Testing Set')
plt.plot(X_TimeArray[train_index:-window], AllDatasets[cryptos[2]][train_index:-window], label = 'Training Set')
plt.plot(X_TimeArray[(-window-1):], AllDatasets[cryptos[2]][(-window-1):], label = f'window ({window} Day)')

plt.title(f'{cryptos[2]} Dataset'), 
plt.xlabel('Time (days)'), plt.ylabel('Price (USD)'), plt.grid(True), plt.legend()
plt.xticks(Full_tick_locations, X_TimeArray[Full_tick_locations], rotation = 60)
plt.savefig(f"{cryptos[2]}_Dataset", dpi=300, bbox_inches = 'tight')


plt.figure()
plt.plot(X_TimeArray[train_index:-window], AllDatasets[cryptos[2]][train_index:-window], label = 'Closing Price')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[2]]['SVR'], label = 'SVR Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[2]]['LSTM'], label = 'LSTM Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[2]]['GRU'], label = 'GRU Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[2]]['CNN'], label = 'CNN Prediction')

plt.title(f'{cryptos[2]} Closing Price Prediction')
plt.xlabel('Time (days)'), plt.ylabel('Price (USD)'), plt.grid(True), plt.legend()
plt.xticks(tick_locations, X_TimeArray[train_index:-window][tick_locations], rotation = 60)
plt.savefig(f"{cryptos[2]}_Predictions", dpi=300, bbox_inches = 'tight')


#--------------------------------------------   BAR CHARTS    -------------------------------------------------------
BarNames = ['SVR', 'LSTM', 'GRU', 'CNN']
colours = ['orange', 'green', 'darkred', 'purple']


plt.figure()
values = [allCrypto[8]['Training Time'], allCrypto[9]['Training Time'], allCrypto[10]['Training Time'], allCrypto[11]['Training Time']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[2]} Training Time')
plt.xlabel('Models')
plt.ylabel('Time (seconds)')
plt.savefig(f"{cryptos[2]}_Training_Time", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[8]['MAPE'], allCrypto[9]['MAPE'], allCrypto[10]['MAPE'], allCrypto[11]['MAPE']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[2]} Mean Absolute Percentage Error (MAPE)')
plt.xlabel('Models')
plt.ylabel('Percentage (%)')
plt.savefig(f"{cryptos[2]}_MAPE", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[8]['MSE'], allCrypto[9]['MSE'], allCrypto[10]['MSE'], allCrypto[11]['MSE']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[2]} Mean Squared Error (MSE)')
plt.xlabel('Models')
plt.ylabel('Price Squared ($USD^2$)')
plt.savefig(f"{cryptos[2]}_MSE", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[8]['R2'], allCrypto[9]['R2'], allCrypto[10]['R2'], allCrypto[11]['R2']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[2]} $R^2$ Measure')
plt.xlabel('Models')
plt.ylabel('No Units')
plt.savefig(f"{cryptos[2]}_R2", dpi=300, bbox_inches = 'tight')

In [ ]:
##########################################  BNB CHARTS  ####################################################### 


#********************************************* GRAPHS *********************************************************

plt.plot(X_TimeArray[:(train_index+1)], AllDatasets[cryptos[3]][:(train_index+1)], label = 'Testing Set')
plt.plot(X_TimeArray[train_index:-window], AllDatasets[cryptos[3]][train_index:-window], label = 'Training Set')
plt.plot(X_TimeArray[(-window-1):], AllDatasets[cryptos[3]][(-window-1):], label = f'window ({window} Day)')

plt.title(f'{cryptos[3]} Dataset'), 
plt.xlabel('Time (days)'), plt.ylabel('Price (USD)'), plt.grid(True), plt.legend()
plt.xticks(Full_tick_locations, X_TimeArray[Full_tick_locations], rotation = 60)
plt.savefig(f"{cryptos[3]}_Dataset", dpi=300, bbox_inches = 'tight')



plt.figure()
plt.plot(X_TimeArray[train_index:-window], AllDatasets[cryptos[3]][train_index:-window], label = 'Closing Price')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[3]]['SVR'], label = 'SVR Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[3]]['LSTM'], label = 'LSTM Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[3]]['GRU'], label = 'GRU Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[3]]['CNN'], label = 'CNN Prediction')

plt.title(f'{cryptos[3]} Closing Price Prediction')
plt.xlabel('Time (days)'), plt.ylabel('Price (USD)'), plt.grid(True), plt.legend(fontsize=9.45,loc='lower right')
plt.xticks(tick_locations, X_TimeArray[train_index:-window][tick_locations], rotation = 60)
plt.savefig(f"{cryptos[3]}_Predictions", dpi=300, bbox_inches = 'tight')


#--------------------------------------------   BAR CHARTS   -------------------------------------------------------
BarNames = ['SVR', 'LSTM', 'GRU', 'CNN']
colours = ['orange', 'green', 'darkred', 'purple']


plt.figure()
values = [allCrypto[12]['Training Time'], allCrypto[13]['Training Time'], allCrypto[14]['Training Time'], allCrypto[15]['Training Time']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[3]} Training Time')
plt.xlabel('Models')
plt.ylabel('Time (seconds)')
plt.savefig(f"{cryptos[3]}_Training_Time", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[12]['MAPE'], allCrypto[13]['MAPE'], allCrypto[14]['MAPE'], allCrypto[15]['MAPE']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[3]} Mean Absolute Percentage Error (MAPE)')
plt.xlabel('Models')
plt.ylabel('Percentage (%)')
plt.savefig(f"{cryptos[3]}_MAPE", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[12]['MSE'], allCrypto[13]['MSE'], allCrypto[14]['MSE'], allCrypto[15]['MSE']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[3]} Mean Squared Error (MSE)')
plt.xlabel('Models')
plt.ylabel('Price Squared ($USD^2$)')
plt.savefig(f"{cryptos[3]}_MSE", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[12]['R2'], allCrypto[13]['R2'], allCrypto[14]['R2'], allCrypto[15]['R2']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[3]} $R^2$ Measure')
plt.xlabel('Models')
plt.ylabel('No Units')
plt.savefig(f"{cryptos[3]}_R2", dpi=300, bbox_inches = 'tight')

In [ ]:
##########################################  USDC CHARTS  ####################################################### 


#*********************************************  GRAPHS *********************************************************

plt.plot(X_TimeArray[:(train_index+1)], AllDatasets[cryptos[4]][:(train_index+1)], label = 'Testing Set')
plt.plot(X_TimeArray[train_index:-window], AllDatasets[cryptos[4]][train_index:-window], label = 'Training Set')
plt.plot(X_TimeArray[(-window-1):], AllDatasets[cryptos[4]][(-window-1):], label = f'window ({window} Day)')

plt.title(f'{cryptos[4]} Coin Dataset'), 
plt.xlabel('Time (days)'), plt.ylabel('Price (USD)'), plt.grid(True), plt.legend(loc=(0.05,0.6))
plt.xticks(Full_tick_locations, X_TimeArray[Full_tick_locations], rotation = 60)
plt.savefig(f"{cryptos[4]}_Dataset", dpi=300, bbox_inches = 'tight')



plt.figure()
plt.plot(X_TimeArray[train_index:-window], AllDatasets[cryptos[4]][train_index:-window], label = 'Closing Price')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[4]]['SVR'], label = 'SVR Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[4]]['LSTM'], label = 'LSTM Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[4]]['GRU'], label = 'GRU Prediction')
plt.plot(X_TimeArray[train_index:-window], y_plot_predvals[cryptos[4]]['CNN'], label = 'CNN Prediction')

plt.title(f'{cryptos[4]} Closing Value Prediction')
plt.xlabel('Time (days)'), plt.ylabel('Price (USD)'), plt.grid(True), plt.legend(loc=(0.65,0.45), fontsize = '9')
plt.xticks(tick_locations, X_TimeArray[train_index:-window][tick_locations], rotation = 60)
plt.savefig(f"{cryptos[4]}_Predictions", dpi=300, bbox_inches = 'tight')


#--------------------------------------------   BAR CHARTS    -------------------------------------------------------
BarNames = ['SVR', 'LSTM', 'GRU', 'CNN']
colours = ['orange', 'green', 'darkred', 'purple']


plt.figure()
values = [allCrypto[16]['Training Time'], allCrypto[17]['Training Time'], allCrypto[18]['Training Time'], allCrypto[19]['Training Time']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[4]} Coin Training Time')
plt.xlabel('Models')
plt.ylabel('Time (seconds)')
plt.savefig(f"{cryptos[4]}_Training_Time", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[16]['MAPE'], allCrypto[17]['MAPE'], allCrypto[18]['MAPE'], allCrypto[19]['MAPE']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[4]} Coin Mean Absolute Percentage Error (MAPE)')
plt.xlabel('Models')
plt.ylabel('Percentage (%)')
plt.savefig(f"{cryptos[4]}_MAPE", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[16]['MSE'], allCrypto[17]['MSE'], allCrypto[18]['MSE'], allCrypto[19]['MSE']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[4]} Coin Mean Squared Error (MSE)')
plt.xlabel('Models')
plt.ylabel('Price Squared ($USD^2$)')
plt.savefig(f"{cryptos[4]}_MSE", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[16]['R2'], allCrypto[17]['R2'], allCrypto[18]['R2'], allCrypto[19]['R2']]
plt.bar(BarNames, values, color = colours)
plt.title(f'{cryptos[4]} Coin $R^2$ Measure')
plt.xlabel('Models')
plt.ylabel('No Units')
plt.savefig(f"{cryptos[4]}_R2", dpi=300, bbox_inches = 'tight')

plt.figure()
values = [allCrypto[16]['R2']]
plt.bar('SVR', values, color = 'orange')
plt.title(f'{cryptos[4]} Coin $R^2$ Measure')
plt.xlabel('Model (SVR)')
plt.ylabel('No Units')
plt.savefig(f"{cryptos[4]}_SVR_R2", dpi=300, bbox_inches = 'tight')

plt.figure()
BarNames = ['LSTM', 'GRU', 'CNN']
values = [allCrypto[17]['R2'], allCrypto[18]['R2'], allCrypto[19]['R2']]
plt.bar(BarNames, values, color = ['green','darkred','purple'])
plt.title(f'{cryptos[4]} Deep Learning Algorithms $R^2$ Measure')
plt.xlabel('Models')
plt.ylabel('No Units')
plt.savefig(f"{cryptos[4]}_LSTM_GRU_CNN_R2", dpi=300, bbox_inches = 'tight')

In [ ]:
# BarNames = ['LSTM', 'GRU', 'CNN']
# values = [-4.82446712435037, -2.16223990596749, -0.783226905933456]
# plt.bar(BarNames, values, color = ['green','darkred','purple'])
# plt.title(f'{cryptos[4]} Deep Learning Algorithms $R^2$ Measure')
# plt.xlabel('Models')
# plt.ylabel('No Units')
# plt.savefig(f"{cryptos[4]}_LSTM_GRU_CNN_R2", dpi=300, bbox_inches = 'tight')

In [ ]:
# plt.figure()
# BarNames = ['SVR', 'LSTM', 'GRU', 'CNN']
# colours = ['orange', 'green', 'darkred', 'purple']
# values = [-67.3217670562895, -112.579292597834, -66.4872934015324, -1.1579009176922]
# plt.bar(BarNames, values, color = colours)
# plt.title(f'Tether $R^2$ Measure')
# plt.xlabel('Models')
# plt.ylabel('No Units')
# plt.savefig("Tether_R2", dpi=300, bbox_inches = 'tight')